In [1]:
import requests

address = " бізон, Солом'янський район, Київ, Україна"
nominatim_url = f"https://nominatim.openstreetmap.org/search?q={address}&format=json"
response = requests.get(nominatim_url)
location = response.json()[0]

print(f"Широта: {location['lat']}, Довгота: {location['lon']}")


Широта: 50.44333105, Довгота: 30.437891675624538


Помилка запиту: 403
Нічого не знайдено.
